In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [6]:
train_out=pd.read_csv("/content/drive/MyDrive/train_out.csv")
train_out.drop(["Unnamed: 0"], axis=1, inplace=True)
train_out.head(2)

,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,2,1,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.000000,0.463330,34.160000,53.996091,0,0.000000,0.20000,0.0,0.041667,0.025000,0.0,0.000000,0.175,0.033333,0.0,1
1,2,2,0.0,0.0,0.0,0,0.0,0.0,0.076923,0.153846,0.429468,15.285714,96.231625,0,0.008475,0.09322,0.0,0.012712,0.018644,0.0,0.084746,0.000,0.033898,82.5,2


In [12]:
train_out.columns

Index(['tier', 'gender', 'following_rate', 'followers_avg_age',
       'following_avg_age', 'max_repetitive_punc',
       'num_of_hashtags_per_action', 'emoji_count_per_action',
       'punctuations_per_action', 'number_of_words_per_action',
       'avgCompletion', 'avgTimeSpent', 'avgDuration', 'avgComments',
       'creations', 'content_views', 'num_of_comments',
       'weekends_trails_watched_per_day', 'weekdays_trails_watched_per_day',
       'slot1_trails_watched_per_day', 'slot2_trails_watched_per_day',
       'slot3_trails_watched_per_day', 'slot4_trails_watched_per_day', 'avgt2',
       'age_group'],
      dtype='object')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
y = train_out['age_group']
X = train_out.drop(['age_group'], axis=1, inplace=False)

In [8]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_sample(X,y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [9]:
train_x,test_x,train_y,test_y=train_test_split(X_res,y_res,test_size=.0625,random_state=42, stratify = y_res)

In [10]:
type(train_x)

numpy.ndarray

In [11]:
type(train_y)

numpy.ndarray

In [13]:
train_X = pd.DataFrame(train_x, columns = ['tier', 'gender', 'following_rate', 'followers_avg_age',
       'following_avg_age', 'max_repetitive_punc',
       'num_of_hashtags_per_action', 'emoji_count_per_action',
       'punctuations_per_action', 'number_of_words_per_action',
       'avgCompletion', 'avgTimeSpent', 'avgDuration', 'avgComments',
       'creations', 'content_views', 'num_of_comments',
       'weekends_trails_watched_per_day', 'weekdays_trails_watched_per_day',
       'slot1_trails_watched_per_day', 'slot2_trails_watched_per_day',
       'slot3_trails_watched_per_day', 'slot4_trails_watched_per_day', 'avgt2'])
train_X.head(2)

,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2
0,2.000000,1.0,0.003630,0.0,0.0,0.000000,0.0,0.0,0.000000,0.111111,0.128572,33.833333,186.497143,0.0,0.001815,0.012704,0.0,0.010889,0.000000,0.000000,0.000000,0.019964,0.001815,221.071429
1,2.717098,1.0,0.007664,0.0,0.0,1.434196,0.0,0.0,0.068295,0.124875,0.128495,13.529996,178.924352,0.0,0.006118,0.107990,0.0,0.000000,0.030562,0.044819,0.012751,0.000000,0.095240,125.282902


In [14]:
test_X = pd.DataFrame(test_x, columns = ['tier', 'gender', 'following_rate', 'followers_avg_age',
       'following_avg_age', 'max_repetitive_punc',
       'num_of_hashtags_per_action', 'emoji_count_per_action',
       'punctuations_per_action', 'number_of_words_per_action',
       'avgCompletion', 'avgTimeSpent', 'avgDuration', 'avgComments',
       'creations', 'content_views', 'num_of_comments',
       'weekends_trails_watched_per_day', 'weekdays_trails_watched_per_day',
       'slot1_trails_watched_per_day', 'slot2_trails_watched_per_day',
       'slot3_trails_watched_per_day', 'slot4_trails_watched_per_day', 'avgt2'])
test_X.head(2)

,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,avgCompletion,avgTimeSpent,avgDuration,avgComments,creations,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2
0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.355810,4.428571,12.665286,0.0,0.000000,0.065041,0.0,0.0,0.011382,0.0,0.000000,0.0,0.056911,0.000000
1,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.341657,6.000000,83.051415,0.0,0.007812,0.007812,0.0,0.0,0.003125,0.0,0.007812,0.0,0.007812,164.833333


In [15]:
train_Y=pd.DataFrame(train_y, columns = ['age_group'])
train_Y.head(2)

,age_group
0,3
1,2


In [16]:
test_Y=pd.DataFrame(test_y, columns = ['age_group'])
test_Y.head(2)

,age_group
0,1
1,1


In [17]:
xgb_clf = XGBClassifier(tree_method = 'gpu_hist', gpu_id=0, objective = "multi:softmax", num_class=4,n_estimators=1000, max_depth=15,learning_rate=0.005)

In [ ]:
xgb_hist = xgb_clf.fit(train_X, train_Y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn import metrics
expected_y  = test_Y
predicted_y = xgb_clf.predict(test_X)
print('Report: ')
print(metrics.classification_report(expected_y, predicted_y))

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/test_age_dataset.csv')
test.drop(["Unnamed: 0","userId"], axis=1, inplace=True)
test.head(2)

In [ ]:
predict = xgb_clf.predict(test)

In [ ]:
df_pred = pd.DataFrame({"prediction" : predict}, columns = ["prediction"]).set_index("prediction")

In [ ]:
def download_preds(df_pred, file_name = 'outliers_4_test_predicted.csv'):
  df_pred.to_csv(file_name)
  from google.colab import files
  files.download(file_name)
download_preds(df_pred, file_name = 'outliers_4_test_predicted.csv')